# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [9]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors = 'coerce')
df_youtube = df_youtube.dropna(subset=['date'])
 

df_youtube.rename(columns = {'video_id':'_id'}, inplace = True)
 
df_youtube = df_youtube.drop_duplicates(subset = '_id', keep = 'last')
 
l_d = df_youtube.to_dict(orient = 'records')
 
import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
 
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [5]:
from pymongo import MongoClient
client = MongoClient("mongo:27017")

In [6]:
db_yt = client.yt
collection_louis_yt = db_yt['videos']

In [7]:
collection_louis_yt.delete_many({})
collection_louis_yt.insert_many(l_d)

ServerSelectionTimeoutError: mongo:27017: [Errno -3] Temporary failure in name resolution

## Question 1  

In [ ]:
req = collection_louis_yt.find({"channel_title":"Apple"})
next(req)

## Question 2

In [ ]:
request = collection_louis_yt.aggregate([
    {"$group": {"_id": "$category_id"}},
    {"$group": {"_id": "category_id", "count": { "$sum": 1 }}}
])
list(request)

## Question 3

In [ ]:
for video in l_d:
  video['tags'] = video['tags'].split('|') 
collection.update_one(
    {"_id": {"$eq":video['_id']}},
    { "$set": {"tags":video['tags']}}
)
req = collection.find()
next(req)

## Question 4

In [ ]:
cur = collection_louis_yt.find({}).sort([("views", -1)])
list(cur)

## Question 5

In [ ]:
query = collection.aggregate([{"$group": {"_id": "$category_id", "views": {"$avg" : "$views"}}}])
list(query)

## Question 6 

In [ ]:
query = collection.aggregate([{"$group": {"_id": "$channel_title", "average_likes": {"$avg" : "$likes"}}}])
list(query)